In [79]:
import urllib.request as req
import json
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

# Catch the data from url

In [80]:
url = "https://www.ptt.cc/bbs/LoL/M.1684235300.A.338.html"
response = requests.get(url)
html = response.text
#request = req.Request(url, headers = {
#    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
#})
#with req.urlopen(request) as response:
#    data = response.read().decode("utf-8")

In [81]:
soup = BeautifulSoup(html, 'html.parser')
allcomments = soup.find_all('div', {'class': 'push'})

# Use array 'temp' to save all comments

In [82]:
temp = []
for i in allcomments:
    tag = i.find('span')
    #判斷是否為None
    if tag is not None:
        text = i.find('span', {'class': 'f3 push-content'}).getText().strip()    
        temp.append(text)

df = pd.DataFrame(temp)
df

,0
0,: BLG 3:0 虐菜屠殺局 十點前沒結束算G2贏
1,: G2會贏
2,: G2會贏，只要贏一把就是G2的巨大勝利
3,: g2 打滿bo5算贏
4,: G2加油點 我把這個月抽輪椅的一單下在你們身上了
...,...
1364,: 怎麼會有人說Deft今年超爛 沒看比賽吧
1365,: 今年DK是以往最強的中野輪流在戳的，Deft打的算不
1366,: 錯了
1367,: 駝今年真的還可以吧


# Analyze the key words from the comments

In [83]:
import jieba
import jieba.analyse
from collections import Counter

In [84]:
df['seg'] = " "
index = 0
for documents in df[0]:
    seg_list = jieba.analyse.extract_tags(documents, 150)
    seg_list = jieba.lcut(documents)
    df['seg'].iloc[index] = seg_list
    index += 1

key = df['seg']

df2 = pd.DataFrame(key)
df2

,seg
0,"[:, , BLG, , 3, :, 0, , 虐菜, 屠殺局, , , , ..."
1,"[:, , G2, 會贏]"
2,"[:, , G2, 會贏, ，, 只要, 贏, 一把, 就是, G2, 的, 巨大, 勝利]"
3,"[:, , g2, , 打滿, bo5, 算贏]"
4,"[:, , G2, 加油, 點, , 我, 把, 這個, 月, 抽輪椅, 的, 一單, ..."
...,...
1364,"[:, , 怎麼, 會, 有人, 說, Deft, 今年, 超爛, , 沒看, 比賽, 吧]"
1365,"[:, , 今年, DK, 是, 以往, 最強, 的, 中野, 輪流, 在, 戳, 的, ..."
1366,"[:, , 錯, 了]"
1367,"[:, , 駝, 今年, 真的, 還, 可以, 吧]"


# Save as a csv file

In [85]:
import csv

with open('LOL2.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['comments', 'keywords'])
       
    for i in range(len(temp)):
        comments_i = temp[i]
        keywords_i = key[i]
        writer.writerow([comments_i, keywords_i])

data = pd.read_csv("LOL2.csv")
data

,comments,keywords
0,: BLG 3:0 虐菜屠殺局 十點前沒結束算G2贏,"[':', ' ', 'BLG', ' ', '3', ':', '0', ' ', '虐菜..."
1,: G2會贏,"[':', ' ', 'G2', '會贏']"
2,: G2會贏，只要贏一把就是G2的巨大勝利,"[':', ' ', 'G2', '會贏', '，', '只要', '贏', '一把', '..."
3,: g2 打滿bo5算贏,"[':', ' ', 'g2', ' ', '打滿', 'bo5', '算贏']"
4,: G2加油點 我把這個月抽輪椅的一單下在你們身上了,"[':', ' ', 'G2', '加油', '點', ' ', '我', '把', '這個..."
...,...,...
1364,: 怎麼會有人說Deft今年超爛 沒看比賽吧,"[':', ' ', '怎麼', '會', '有人', '說', 'Deft', '今年',..."
1365,: 今年DK是以往最強的中野輪流在戳的，Deft打的算不,"[':', ' ', '今年', 'DK', '是', '以往', '最強', '的', '..."
1366,: 錯了,"[':', ' ', '錯', '了']"
1367,: 駝今年真的還可以吧,"[':', ' ', '駝', '今年', '真的', '還', '可以', '吧']"


# Add up the keywords

In [86]:
#字典序排序資料
subdata = data[['comments', 'keywords']]
output_series = subdata.groupby(['comments'])['keywords'].apply(lambda x: ','.join(x)).reset_index()

#去掉comments 和 keywords分段標號跟冒號
output_series['comments'] = output_series['comments'].str.replace(":", "")
output_series['keywords'] = output_series['keywords'].str.replace("', '", " ")
output_series['keywords'] = output_series['keywords'].str.replace(":", "")
output_series

,comments,keywords
0,blg及格了 可以下去 geng穩了,[' blg 及格 了 可以 下去 geng 穩 了']
1,白癡摳,[' 白 癡 摳']
2,"""打得非常好"" ""非常強""","[' "" 打 得 非常 好 "" "" 非常 強 ""']"
3,-1445,[' - 1445']
4,.............,[' .............']
...,...,...
1305,黑甲一個輔助都殺不掉笑死,[' 黑甲 一個 輔助 都 殺 不 掉 笑 死']
1306,黑甲快贏1k了,[' 黑甲 快贏 1k 了']
1307,黑甲怎麼都沒人拿,[' 黑甲 怎麼 都 沒人 拿']
1308,點數毆印雞兔,[' 點數 毆 印雞 兔']


In [87]:
from sklearn.feature_extraction.text import CountVectorizer

docs = output_series['keywords'].tolist()
vec = CountVectorizer()
X = vec.fit_transform(docs)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df

C:\Users\venchi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



,032,0g2,100,1000,1133,12,1445,15,150,16,...,黑甲切爛,黑甲會,黑進,點啊,點戲,點數,點點點,鼓勵,龍坑,龐克頭
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1307,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
coocc = df.T.dot(df)
coocc

import plotly.express as px
px.imshow(coocc)

result = coocc.values.flatten()

fig = px.histogram(result)
fig.show()

#各詞彙出現約1-5次不等

# Draw the heat map

In [ ]:
filter_num = 15
getid = coocc.index
important = pd.DataFrame(columns = getid).T
important['relation'] = ""
subcoocc = coocc
dropid = []

for i in range(len(coocc)):
    result = coocc[coocc.iloc[i] > filter_num].index.tolist()    
    if(len(result) > 0):
        important['relation'].loc[getid[i]] = coocc[coocc.iloc[i] > filter_num].index.tolist()
    else:
        dropid.append(i)
        important = important.drop([getid[i]])
        subcoocc = subcoocc.drop([getid[i]]) 

In [ ]:
subcoocc

In [ ]:
#取 col 內容
subcoocc.columns[dropid]

In [ ]:
#將 row 也只保留與 col 相同內容之數據
subcoocc.drop(subcoocc.columns[dropid], axis=1, inplace=True)
subcoocc

In [ ]:
#因為 row=col 的詞語內容相同，在熱點圖上數據會過於突出，依比例進行調整
for i in range(len(subcoocc)):
    subcoocc.iloc[i, i] *= -0.05
subcoocc

In [ ]:
#draw the heat map
px.imshow(subcoocc)